In [20]:
import torch,torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np

In [23]:
def minstIID(dataset, num_users):
    num_images = int(len(dataset)/num_users)
    users_dict, indeces = {}, list(range(len(dataset)))
    for i in range(num_users):
        np.random.seed(i) # to expect the same random value each time of running  the code
        users_dict[i] = set(np.random.choice(indeces,num_images,replace=False))
        indeces = list(set(indeces) - users_dict[i]) # remove used indeces from main indeces
    return users_dict
                            

In [24]:
# Load Data
def load_dataset(num_users):
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,),(0.3081,))])
    train_dataset = torchvision.datasets.MNIST(root = "./data", train = True, transform = transform, target_transform = None, download = True)
    test_dataset = torchvision.datasets.MNIST(root = "./data", train = False, transform = transform, target_transform = None, download = True)
    train_group = minstIID(train_dataset, num_users)
    test_group = minstIID(test_dataset, num_users)
    return train_dataset, test_dataset, train_group, test_group


In [10]:
#train_set

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )

In [11]:
#test_set

Dataset MNIST
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )

In [28]:
class FedDataset(Dataset):
    def __init__(self,dataset,indx):
        self.dataset = dataset
        self.indx = [int(i) for i in indx]
    
    def __len__(self):
        return len(self.indx)
    
    def __getitem__(self,item):
        images,label = self.dataset[self.indx[item]]
        #return torch.tensor(images),torch.tensor(label)
        return torch.tensor(images).clone().detach(), torch.tensor(label).clone().detach()
        

In [25]:
def getActualImgs(dataset, indeces, batch_size):
    return DataLoader(FedDataset(dataset,indeces), batch_size=batch_size, shuffle=True)